# Hierarchical Gibbs using cuqipy conditional distributions

In the following notebook we showcase the conditional distribution framework in cuqipy with the ultimate aim of implementing a succinctly written Gibbs sampler that samples the posterior related to an inverse problem where the the variance of the noise and prior are modeled as hyper-parameters.

The example is based on the problem used to demonstrate Algorithm 5.1 in [1], but before we get there let us go through the basics of conditional distributions in cuqipy


[1] *Bardsley, Johnathan. 2018. Computational Uncertainty Quantification for Inverse Problems. SIAM, Society for Industrial and Applied Mathematics.*

In [ ]:
# First we need to import cuqi
import sys
import numpy as np
import cuqi
import matplotlib.pyplot as plt

## Distributions
The first thing we can do is define a simple normal distribution, e.g.,

$$ X = \mathcal{N}(0,1^2) $$

This is done using the following syntax

In [ ]:
# A simple normal distribution with mean 5 and std 1
X = cuqi.distribution.Normal(mean=0,std=1)

Distributions in cuqipy have commonly used methods that one might expect like *pdf*, *logpdf*, *sample* etc. More information on the distribtion can be found by typing `help(y)`. Let us try to generate a single sample as follows.

In [ ]:
X.sample()

## Conditional distributions

In cuqipy defining conditional distributions is simple. Assume we are interested in defining the Normal distribution condtioned on the standard deviation, e.g.

$$ X_2 | \mathrm{std} = \mathcal{N}(0,\mathrm{std}^2) $$

This can simply be achieved by *omitting* the keyword argument for the standard deviation as shown in the following code

In [ ]:
X2 = cuqi.distribution.Normal(mean=0)

Because $X_2$ is a conditional distribution, we cannot evaluate the logpdf or sample it directly without specifying the value of the conditional parameter (the standard deviation in this case). Hence the first line in the code cell below would fail.

However, we can specify the conditional parameter using the "call" syntax, i.e., `X2(std=2)` to specify the value of the standard deviation in the conditional distribution as shown below.

In [ ]:
# X2.sample() #This code would fail
X2(std=2).sample()

In general one may need more flexibility than simply conditioning directly on the attributes of the distribution. Let us assume we want to condition on the variance - denoted d - rather than the standard deviation of the normal distribution, i.e.

$$ X_3 | d = \mathcal{N}(0,d) $$

In cuqipy this is handled by *lambda* functions as follows.

In [ ]:
X3 = cuqi.distribution.Normal(mean=0,std=lambda d: np.sqrt(d))
X3(d=2).sample()

What actually happens behind the scenes is that writing `X3(d=2)` defined a new cuqi distribution, where the standard deviation is defined by evaluating the lambda function. This can be seen by storing the new distribution as follows.

In [ ]:
X4 = X3(d=2)
X4.std

One can even go crazy and define lambda functions for all attributes e.g.

In [ ]:
#Functions for mean and std with various (shared) inputs
mean = lambda sigma,gamma: sigma+gamma
std  = lambda delta,gamma: np.sqrt(delta+gamma)

z = cuqi.distribution.Normal(mean,std)
Z = z(sigma=3,delta=5,gamma=-2)

Z.sample()

## Hierarchical Gibbs sampler

In the following we aim to implement a Hierarchical Gibbs sampler for a posterior related to an inverse problem based on algorithm 5.1 in [1]. For completeness we state the problem and posterior first.

We are interested in the inverse problem
$$ \mathbf{b} = \mathbf{A}\mathbf{x}+\mathbf{e}, \quad \mathbf{e}\sim\mathcal{N}(\mathbf{0},\lambda^{-1}\mathbf{I}_m), \mathbf{x}\sim\mathcal{N}(\mathbf{0},\delta^{-1}\mathbf{I}_n)$$
where $\mathbf{A}\in\R^{m\times n}$, $\mathbf{x}\in\R^n$ and $\mathbf{b}\in\R^m$ and where $\lambda,\delta\in\R_+$ are considered hyper-parameters.

The posterior is given by
$$ p(\mathbf{x}|\mathbf{b},\lambda,\delta) \propto p(\mathbf{b}|\mathbf{x},\lambda)p(\mathbf{x}|\delta), $$

where the likelihood pdf is
$$ p(\mathbf{b}|\mathbf{x},\lambda) = \left(\frac{\lambda}{2\pi}\right)^{m/2}\exp\left( -\frac{\lambda}{2}\| \mathbf{A}\mathbf{x}-\mathbf{b}\|_2^2 \right), $$
and prior pdf is
$$ p(\mathbf{x}|\delta) = \left(\frac{\delta}{2\pi}\right)^{n/2}\exp\left( -\frac{\delta}{2}\|\mathbf{x}\|_2^2\right) $$

It is commonly the case that hyper-parameters are defined as Gamma distributions, e.g.,
$$ p(\lambda)\propto \lambda^{\alpha-1}\exp(\beta \lambda) $$
$$ p(\delta) \propto \delta^{\alpha-1}\exp(\beta \delta) $$

In cuqi the (perhaps) most obvious way of defining this problem would be to write something like the code below.

In [ ]:
#Load model + data from testproblem
TP = cuqi.testproblem.Deconvolution() #Default values
A = TP.model
b = TP.data

# Extract dimensions
n = A.domain_dim
m = A.range_dim

# Parameters for hyper-parameters
alpha = 1
beta = 1e-4

# Hyper-parameters
d = cuqi.distribution.Gamma(shape=alpha,rate=beta)
l = cuqi.distribution.Gamma(shape=alpha,rate=beta)

#Prior
x = cuqi.distribution.GaussianCov(mean=np.zeros(n),cov=lambda d: 1/d)

# Likelihood
L = cuqi.distribution.GaussianCov(mean=A, cov = lambda l: 1/l)

Unfurtunately there are no no hierachical samplers implemented in cuqipy yet. Hence code like (**not final syntax**)

`cuqi.problem.BayesianProblem(likelihood=L,prior=x,data=b,hyper_para = [d,l])`

or 

`cuqi.sampler.Gibbs(likelihood=L,prior=x,data=b,hyper_para = [d,l])`

is not supported yet.

## Implementing a Gibbs sampler in cuqipy
Instead for now we would have to implement our own Gibbs sampler in cuqipy while taking advantage of the conditional distribution framework. In [1] a Gibbs sampler is proposed for exactly the problem we defined with $\mathbf{L}=\mathbf{I}$ and we restate the Gibbs sampler here.

### Algorithm 5.1. The Hierarchical Gibbs Sampler.

0. Initialize $\left(\lambda_{0}, \delta_{0}\right), \mathbf{x}^{0}=\left(\lambda_{0} \mathbf{A}^{T} \mathbf{A}+\delta_{0} \mathbf{L}\right)^{-1} \lambda_{0} \mathbf{A}^{T} \mathbf{b}$, set $k=1$, define $k_{\text {total }}.$
1. Compute $\left(\lambda_{k}, \delta_{k}\right) \sim p\left(\lambda, \delta \mid \mathbf{b}, \mathbf{x}^{k-1}\right)$ as follows.
    - a) Compute $\lambda_{k} \sim \Gamma\left(M / 2+\alpha_{\lambda}, \frac{1}{2}\left\|\mathbf{A} \mathbf{x}^{k-1}-\mathbf{b}\right\|^{2}+\beta_{\lambda}\right)$.
    - b) Compute $\delta_{k} \sim \Gamma\left(\bar{N} / 2+\alpha_{\delta}, \frac{1}{2}\left(\mathbf{x}^{k-1}\right)^{T} \mathbf{L} \mathbf{x}^{k-1}+\beta_{\delta}\right)$.
2. Compute $\mathbf{x}^{k} \sim \mathcal{N}\left(\left(\lambda_{k} \mathbf{A}^{T} \mathbf{A}+\delta_{k} \mathbf{L}\right)^{-1} \lambda_{k} \mathbf{A}^{T} \mathbf{b},\left(\lambda_{k} \mathbf{A}^{T} \mathbf{A}+\delta_{k} \mathbf{L}\right)^{-1}\right)$
3. If $k=k_{\text {total }}$ stop, otherwise, set $k=k+1$ and return to Step 1 .

To implement the above-mentioned sampler we would need to define the conditional distributions in steps 1a, 1b and 2. This is simply carried out as shown below.

In [ ]:
#Matricies
A = A.get_matrix()
L = np.eye(n)

# Define hyperpriors
l = cuqi.distribution.Gamma(shape=m/2+alpha,rate=lambda x: .5*np.linalg.norm(A@x-b)**2+beta)
d = cuqi.distribution.Gamma(shape=n/2+alpha,rate=lambda x: .5*x.T@(L@x)+beta)

# Define prior
mean_func = lambda l,d: np.linalg.solve(l*A.T@A+d*L,l*A.T@b)
cov_func  = lambda l,d: np.linalg.inv(l*A.T@A+d*L)
x = cuqi.distribution.GaussianCov(mean=mean_func,cov=cov_func)

The Gibbs sampler is then implemented as follows.

In [ ]:
# Example from Johns book. Algorithm 5.1
n_samp = 1000

# Preallocate sample vectors
ls = np.zeros(n_samp+1)
ds = np.zeros(n_samp+1)
xs = np.zeros((n,n_samp+1))

# Initial parameters
ls[0] = 1/0.05; ds[0]=1/0.02
xs[:,0] = x(l=ls[0],d=ds[0]).mean

# Gibbs sampler
for k in range(n_samp):

    #Sample hyperparameters conditioned on x
    ls[k+1] = l(x=xs[:,k]).sample()
    ds[k+1] = d(x=xs[:,k]).sample()

    # Sample x conditioned on l,d
    xs[:,k+1] = x(l=ls[k+1],d=ds[k+1]).sample()

Using the cuqi `Samples` and `Geometry` classes we can store the samples and plot e.g. the chains of the hyperparameters.

In [ ]:
hp_s = cuqi.samples.Samples(np.vstack((ls,ds)),geometry=cuqi.geometry.Discrete(["lambda","delta"])) #Discrete geometry
x_s  = cuqi.samples.Samples(xs)                                                                     #Default geometry

In [ ]:
plt.subplot(121); hp_s.plot_chain(0); plt.title("lambda chain")
plt.subplot(122); hp_s.plot_chain(1); plt.title("delta chain");

In [ ]:
x_s.plot_ci(95,exact=TP.exactSolution)